In [4]:
# import statements
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
import pandas as pd

In [5]:
# Iniciando a sessão do Spark
spark = SparkSession.builder.appName("spark-read-imdb").getOrCreate()

In [6]:
# Lendo os arquivos
df_titles = spark.read.csv('data/title_basics.tsv', header=True, inferSchema=True, sep='\t')
df_ratings = spark.read.csv('data/title_ratings.tsv', header=True, inferSchema=True, sep='\t')

# Criando um dataframe único fazendo left join entre as 2 bases pela coluna "tconst"
df_join = df_titles.join(df_ratings, on=["tconst"], how="left")

# Registra o dataframe como uma tabela temporária
df_join.createOrReplaceTempView("temp_imdb")

# Aumenta o limite de tamanho de exibição de colunas
spark.conf.set("spark.sql.columnNameOfInference.truncated", False)

# Aumenta o limite de tamanho da saída do Spark
spark.conf.set("spark.debug.maxToStringFields", 100000)


In [136]:
# Executa uma consulta usando Spark SQL
query = "select * from temp_imdb limit 10"
result = spark.sql(query)
result.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|          5.7|    1809|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|          6.0|     233|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|          6.5|    1560|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|  

In [137]:
# Questão 4: Quantos filmes (incluindo os da televisão) foram lançados no ano de 2015? --OK
query = "select count(*) as qtde from temp_imdb where startYear = 2015 and titleType IN('movie','tvMovie')"
result = spark.sql(query)
result.show()

+-----+
| qtde|
+-----+
|19987|
+-----+



In [138]:
# Questão 5: Qual o gênero de títulos mais frequente? --OK
query = "select genres, count(*) as frequency from temp_imdb group by 1 order by 2 desc limit 1"
result = spark.sql(query)
result.show()

+------+---------+
|genres|frequency|
+------+---------+
| Drama|   880649|
+------+---------+



In [26]:
# Questão 6: Qual o gênero com a melhor nota média de títulos? --OK (History??)
df_explode = df_join.select("tconst", "averageRating", explode(split(df_join["genres"], ",")).alias("genero"))
df_explode.createOrReplaceTempView("temp_teste")
#df_explode.show()

query = "select genero, round(avg(averageRating), 2) as max_rating from temp_teste group by 1 order by 2 desc limit 3"

# #query = "select genres, avg(averageRating) as max_rating from temp_imdb group by 1 order by 2 desc limit 1"
result = spark.sql(query)
result.show()

+-----------+----------+
|     genero|max_rating|
+-----------+----------+
|    History|      7.35|
|Documentary|      7.24|
|  Biography|      7.18|
+-----------+----------+



In [140]:
# Questão 7: Qual o vídeo game do gênero aventura mais bem avaliado em 2020? --OK
query = """ 
        select primaryTitle, averageRating 
        from temp_imdb 
        where startYear = 2020 
            AND genres LIKE '%Adventure%' 
            AND titleType = 'videoGame' 
            order by 2 desc limit 1
        """
result = spark.sql(query)
result.show()

+---------------+-------------+
|   primaryTitle|averageRating|
+---------------+-------------+
|Half-Life: Alyx|          9.5|
+---------------+-------------+



In [141]:
# Questão 9: Quantos títulos de filmes diferentes existem? Use df_titles.select('primaryTitle').distinct().count() --OK
#df_titles.select('primaryTitle').distinct().count() # --> Outro jeito de fazer via API de Data Frame
query = "select count(distinct primaryTitle) from temp_imdb"
result = spark.sql(query)
result.show()

+----------------------------+
|count(DISTINCT primaryTitle)|
+----------------------------+
|                     3931670|
+----------------------------+



In [142]:
# Questão 10: Qual a duração média dos filmes com conteúdo adulto? --OK
query = "select round(avg(runtimeMinutes), 3) as duracao_media from temp_imdb where isAdult = 1"
result = spark.sql(query)
result.show()

+-------------+
|duracao_media|
+-------------+
|       92.799|
+-------------+



In [129]:
# Questão 11: Quantos filmes têm o título atual (“primary”) diferente do título original? Use uma combinação de filter, e count(). --OK
query = "select count(*) as tot from temp_imdb where primaryTitle <> originalTitle"
result = spark.sql(query)
result.show()

+------+
|   tot|
+------+
|125056|
+------+



In [130]:
# Questão 12: Qual o filme que tem o nome mais longo? --OK
query = "select tconst, length(primaryTitle) as tamanho from temp_imdb order by 2 desc limit 1"
result = spark.sql(query)
result.show()

+----------+-------+
|    tconst|tamanho|
+----------+-------+
|tt12985206|    419|
+----------+-------+



In [131]:
# Questão 13: Qual filme tem a maior quantidade de votos? --OK
query = "select tconst, sum(numVotes) as qtde_votos from temp_imdb group by 1 order by 2 desc limit 1"
result = spark.sql(query)
result.show()

+---------+----------+
|   tconst|qtde_votos|
+---------+----------+
|tt0111161|   2449517|
+---------+----------+



In [132]:
# Questão 15: Qual é a menor nota média de um filme? --OK
query = "select min(averageRating) as min_nota_media from temp_imdb order by 1 limit 1"
result = spark.sql(query)
result.show() 

+--------------+
|min_nota_media|
+--------------+
|           1.0|
+--------------+

